<a href="https://colab.research.google.com/github/Roterdamjr/ControleFinanceiro/blob/main/Controle_Financeiro_Python_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biblioteca

> Adicionar aspas



In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.2 MB/s eta 0:00:00


In [75]:
import pandas as pd
import numpy as np
from io import BytesIO
import yfinance as yf

def load_data(excel_filepath):

    df_cadastro_original_acoes = fn_busca_aba_excel(excel_filepath,'Cadastro Original Acoes')
    df_historico = fn_busca_aba_excel(excel_filepath,'Historico')
    df_cadastro_acoes = fn_busca_aba_excel(excel_filepath,'Cadastro Acoes')
    df_cadastro_fi = fn_busca_aba_excel(excel_filepath,'Cadastro FI')
    df_cadastro_recebiveis = fn_busca_aba_excel(excel_filepath,'Cadastro Recebiveis')
    df_proporcoes_dh_dp =pd.read_excel(excel_filepath, sheet_name='Porporcoes DH_DP',  engine='openpyxl' )
    ########## aportes ###############
    df_aportes = pd.read_excel(excel_filepath, sheet_name='Aportes',  engine='openpyxl' )

    aporte_dh = df_aportes['Acoes'].iloc[0] if not df_aportes['Acoes'].empty else 0.0
    aporte_fii = df_aportes['FI'].iloc[0] if not df_aportes['FI'].empty else 0.0
    aporte_recebiveis = df_aportes['Recebiveis'].iloc[0] if not df_aportes['Recebiveis'].empty else 0.0

    return (df_proporcoes_dh_dp, df_cadastro_original_acoes,
            df_historico, df_cadastro_acoes, df_cadastro_fi,
            df_cadastro_recebiveis, aporte_dh, aporte_fii)

def fn_busca_aba_excel(excel_filepath,nome_aba ):
    df_aba = pd.read_excel(
        excel_filepath,
        sheet_name=nome_aba,
        engine='openpyxl'
    )

    df_aba = df_aba.dropna(axis=1,how='all')
    df_aba.set_index('Ativo', inplace=True)
    return df_aba

def fn_busca_pm_quantidade(df):

  #recebe dados de uma ativo e deveolve preco médio, quantidade

  preco_medio, qtde_total = 0.0, 0

  for indice, linha in df.iterrows():
      valor = linha['Valor']
      qtde = linha['Quant']
      operacao = linha['Operação']

      if operacao == 'C':
          preco_medio = (preco_medio * qtde_total + valor * qtde) / (qtde_total + qtde)
          qtde_total += qtde
      elif operacao == 'V':
          qtde_total -= qtde
      elif operacao == 'A':
          preco_medio = (preco_medio * qtde_total) / (qtde_total - qtde)
          qtde_total -= qtde
      elif operacao == 'D':
          preco_medio = (preco_medio * qtde_total) / (qtde_total + qtde)
          qtde_total += qtde

  return qtde_total,preco_medio

def fn_calc_metricas(nome_ativo,
                     df_historico,
                     df_alocacao,
                     df_cotacoes ):

  ##############################
  # calcula quantidade, preco_medio, cotacao ,
  # patrimonio, resultado, rentabilidade, alocacao
  ##############################
  cotacao = df_cotacoes[nome_ativo]

  if nome_ativo in df_historico.index:
    df_ativo = df_historico.loc[nome_ativo]
    quantidade, preco_medio = fn_busca_pm_quantidade(df_ativo)

    alocacao = df_alocacao.loc[nome_ativo]['Alocacao']
    patrimonio = quantidade * cotacao
    resultado = patrimonio - quantidade * preco_medio
    rentabilidade = resultado / (quantidade * preco_medio)
  else:
      quantidade = 0
      preco_medio= 0
      alocacao = 0
      patrimonio = 0
      resultado = 0
      rentabilidade = 0

  return [int(quantidade),
          float(round(preco_medio, 2)),
          float(round(cotacao, 2)) ,
          float(round(patrimonio, 2)),
          float(round(resultado, 2)),
          float(round(rentabilidade, 2)),
          float(round(alocacao, 3))]

def fn_gera_carteira( p_df_historico, p_df_cadastro, p_df_cotacoes, p_aporte):
  df_carteira = pd.DataFrame([fn_calc_metricas(ativo, p_df_historico, p_df_cadastro, p_df_cotacoes)
                              for ativo in p_df_cadastro.index],
                          columns=['Quantidade', 'Preço Médio', 'Cotação', 'Patrimônio', 'Resultado', 'Rentabilidade', 'Alocacao'],
                          index=p_df_cadastro.index)

  # Totais
  patrimonio_total, resultado_total = df_carteira['Patrimônio'].sum(), df_carteira['Resultado'].sum()
  # Aportar
  df_carteira['Aportar'] = ( df_carteira['Alocacao'] * (patrimonio_total + p_aporte) - df_carteira['Patrimônio'] )

  return df_carteira

def fn_busca_cotacoes(p_ativos):
  df_cotacoes = pd.DataFrame()
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
  df_cotacoes.index = df_cotacoes.index.str.replace('.SA', '', regex=False)
  return df_cotacoes

def fn_object_para_numerico(df,nome_coluna):

#    coluna_limpa = df[nome_coluna].astype(str).str.replace('.', '', regex=False)
#    coluna_limpa = coluna_limpa.str.replace(',', '.', regex=False)
 #   df[nome_coluna] = pd.to_numeric(coluna_limpa, errors='coerce')

    df[nome_coluna] = df[nome_coluna].str.replace('.', '', regex=False)
    df[nome_coluna] = df[nome_coluna].str.replace(',', '.', regex=False).astype(float)


def fn_percet_para_numerico(df, nome_coluna):

  coluna_limpa = (
          df[nome_coluna]
          .astype(str)
          .str.replace('%', '', regex=False)
          .str.replace(',', '.', regex=False)
      )

  df['Alocacao'] = pd.to_numeric(coluna_limpa, errors='coerce') / 100




# Teste

In [ ]:
df_historico, df_cadastro_acoes, df_cadastro_fi, df_cadastro_recebiveis, aporte_dh, aporte_fii = load_data('/content/Controle Financeiro.xlsx' )

lista_de_ativos = df_cadastro_acoes.index.tolist()
lista_de_ativos.extend(df_cadastro_fi.index.tolist())
lista_de_ativos.extend(df_cadastro_recebiveis.index.tolist())
df_cotacoes = fn_busca_cotacoes(lista_de_ativos)

/tmp/ipython-input-2218348264.py:92: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
[*********************100%***********************]  32 of 32 completed


In [ ]:
#def fn_calc_metricas(nome_ativo,
#                     df_historico,
#                     df_alocacao,
#                     df_cotacoes ):
nome_ativo='CPSH11'
df_alocacao= df_cadastro_fi
  ##############################
  # calcula quantidade, preco_medio, cotacao ,
  # patrimonio, resultado, rentabilidade, alocacao
  ##############################
cotacao = df_cotacoes[nome_ativo]

if nome_ativo in df_historico.index:
  df_ativo = df_historico.loc[nome_ativo]
  quantidade, preco_medio = fn_busca_pm_quantidade(df_ativo)

  alocacao = df_alocacao.loc[nome_ativo]['Alocacao']
  patrimonio = quantidade * cotacao
  resultado = patrimonio - quantidade * preco_medio
  rentabilidade = resultado / (quantidade * preco_medio)
else:
    alocacao = 0
    patrimonio = 0
    resultado = 0
    rentabilidade = 0

# Alocação Açoes

# Geração de carteiras

In [76]:
df_proporcoes_dh_dp, df_cadastro_original_acoes, \
df_historico, df_cadastro_acoes, df_cadastro_fi, \
df_cadastro_recebiveis, aporte_dh, aporte_fii = load_data('/content/Controle Financeiro.xlsx' )

####################################
#     Download de Cotações
####################################
lista_de_ativos = df_cadastro_acoes.index.tolist()
lista_de_ativos.extend(df_cadastro_fi.index.tolist())
lista_de_ativos.extend(df_cadastro_recebiveis.index.tolist())
df_cotacoes = fn_busca_cotacoes(lista_de_ativos)

/tmp/ipython-input-3252115914.py:111: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
[*********************100%***********************]  32 of 32 completed


In [77]:
####################################
#     Geração das carteiras
####################################
df_carteira_acoes = fn_gera_carteira  (df_historico, df_cadastro_acoes, df_cotacoes, aporte_dh)
df_carteira_acoes['Preco Justo'] = df_cadastro_acoes['Preco Justo']

df_carteira_fi = fn_gera_carteira  (df_historico, df_cadastro_fi, df_cotacoes, aporte_fii)
aporte_recebiveis=30000
df_carteira_recebiveis = fn_gera_carteira  (df_historico, df_cadastro_recebiveis, df_cotacoes, aporte_recebiveis)

In [79]:
df_carteira_acoes

,Quantidade,Preço Médio,Cotação,Patrimônio,Resultado,Rentabilidade,Alocacao,Aportar,Preco Justo
Ativo,,,,,,,,,
ABCB4,670,19.94,24.45,16381.50,3018.67,0.23,0.081,3274.54563,25.00
ALOS3,430,22.07,28.35,12190.50,2701.53,0.28,0.023,-6609.15371,27.00
BBAS3,480,26.26,22.47,10785.60,-1818.59,-0.14,0.000,-10785.60000,20.00
BBDC4,900,13.44,19.65,17685.00,5593.00,0.46,0.068,-1183.62836,20.00
CSMG3,300,22.01,41.63,12489.00,5887.29,0.89,0.081,7167.04563,25.00
CXSE3,1300,14.10,16.19,21047.00,2722.46,0.15,0.108,5161.06084,17.50
EGIE3,90,41.16,30.64,2757.60,-946.68,-0.26,0.029,4279.74967,42.00
FLRY3,300,12.95,16.05,4815.00,931.00,0.24,0.029,2222.34967,14.50
ITSA4,1636,9.95,12.33,20171.88,3900.73,0.24,0.066,-4155.84282,11.00


Carteira de Ações

Carteira de Recebíveis

# Exportação

In [ ]:

df = df_carteira
# -----------------------------------------------------

# 2. Defina o nome do arquivo e crie o ExcelWriter
nome_arquivo = 'relatorio_formatado.xlsx'
writer = pd.ExcelWriter(nome_arquivo, engine='xlsxwriter')

sheet_name = 'Relatório de Investimentos'

# Exporte o DataFrame para a aba, incluindo o índice (Ativo)
# startrow=1 (começa na linha 2), header=False, index=True (inclui a coluna de índice)
df.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False, index=True)

# 3. Acesse o workbook e a worksheet
workbook = writer.book
worksheet = writer.sheets[sheet_name]

# 4. Defina os formatos de cores e números
# Cores RGB em hexadecimal
COR_CINZA = '#A9A9A9'
COR_BEGE_CLARO = '#F5F5DC'
COR_VERDE = '#90EE90'
COR_BEGE_PADRAO = '#F0EAD6'

# Formatos de números (Criação antecipada)
formato_moeda = workbook.add_format({'num_format': 'R$ #,##0.00'})
formato_porcentagem = workbook.add_format({'num_format': '0.00%'})

# Formato do Cabeçalho (Cinza)
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': COR_CINZA,
    'border': 1
})

# Formato da coluna 'Ativo' (Coluna do Índice)
ativo_format = workbook.add_format({'fg_color': COR_BEGE_CLARO, 'border': 1})

# Formato da coluna 'A Aportar' (Verde com formato Moeda)
aportar_format = workbook.add_format({'fg_color': COR_VERDE, 'border': 1, 'num_format': 'R$ #,##0.00'})

# Formato das Colunas Restantes (Bege Padrão)
resto_format = workbook.add_format({'fg_color': COR_BEGE_PADRAO, 'border': 1})
resto_moeda_format = workbook.add_format({'fg_color': COR_BEGE_PADRAO, 'border': 1, 'num_format': 'R$ #,##0.00'})
resto_percent_format = workbook.add_format({'fg_color': COR_BEGE_PADRAO, 'border': 1, 'num_format': '0.00%'})


# 5. Aplique os Formatos
# O cabeçalho inclui o nome do índice (Ativo) na primeira coluna (Coluna 0).
colunas_excel = ['Ativo'] + list(df.columns.values)

# Escreva o Cabeçalho (na linha 1, pois o DataFrame começa na linha 2 com startrow=1)
for col_num, value in enumerate(colunas_excel):
    worksheet.write(0, col_num, value, header_format)
    # Adiciona a formatação de tamanho nas colunas
    if value in ['Ativo', 'Tipo']:
        worksheet.set_column(col_num, col_num, 10)
    elif value in ['Quantidade', 'Preço Médio', 'Cotação']:
        worksheet.set_column(col_num, col_num, 15)
    else:
        worksheet.set_column(col_num, col_num, 12)

# Mapeamento de colunas para o formato específico
formatos_especificos = {
    'Ativo': ativo_format,
    'Preço Médio': resto_moeda_format,
    'Cotação': resto_moeda_format,
    'Patrimônio': resto_moeda_format,
    'Resultado': resto_moeda_format,
    'Rentabilidade': resto_percent_format,
    'Alocacao': resto_percent_format,
    'A Aportar': aportar_format,
}


num_linhas = len(df)
# Itera sobre todas as linhas do DataFrame (começando da linha 1 do Excel)
# row = Linha do Excel (0 é o cabeçalho)
for row in range(1, num_linhas + 1):

    # Itera sobre todas as COLUNAS do Excel (incluindo o índice)
    for col_num, col_name in enumerate(colunas_excel):

        # 1. Obter o valor
        if col_name == 'Ativo':
            # Valor do índice (Ativo)
            valor = df.index[row - 1]
        else:
            # Valor da coluna de dados
            valor = df.iloc[row - 1][col_name]

        # 2. Obter o formato (Se não tiver formato específico, usa o resto_format)
        formato = formatos_especificos.get(col_name, resto_format)

        # 3. Escrever na célula
        worksheet.write(row, col_num, valor, formato)

# 6. Salve o arquivo
writer.close()

print(f"Arquivo '{nome_arquivo}' exportado com sucesso com a formatação de cores e números!")

Arquivo 'relatorio_formatado.xlsx' exportado com sucesso com a formatação de cores e números!
